### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5485,2023-11-02,Brasil Nbb,11:00,Paulistano,São Paulo,2.30,1.56,154.5,1.83,1.83,3.5,1.83,1.83
5486,2023-11-02,Brasil Nbb,19:30,Unifacisa,Bauru,1.42,2.70,152.5,1.83,1.83,-5.5,1.83,1.83
5487,2023-11-02,Eua Nba,22:00,Utah Jazz,Orlando Magic,1.91,1.91,222.5,1.91,1.91,1.5,1.91,1.91
5488,2023-11-02,Europa Euroliga,14:30,Crvena zvezda,Bayern,1.38,3.15,157.5,1.91,1.91,-5.5,1.83,2.00
5489,2023-11-02,Europa Euroliga,14:45,Fenerbahce,Olympiakos,1.54,2.65,152.5,1.91,1.91,-4.5,1.95,1.87
5490,2023-11-02,Europa Euroliga,15:00,Zalgiris Kaunas,Lyon-Villeurbanne,1.30,3.65,161.5,1.91,1.91,-7.5,1.91,1.91
5491,2023-11-02,Europa Euroliga,16:30,Milano,Monaco,1.87,1.95,160.5,1.91,1.91,-1.5,1.95,1.87
5492,2023-11-02,Europa Euroliga,16:30,Valencia,Alba Berlin,1.22,4.50,160.5,1.91,1.91,-9.5,1.95,1.87
5493,2023-11-02,Europa Eurocopa,16:30,London Lions,Hapoel Tel-Aviv,1.87,1.95,172.5,1.91,1.91,-1.5,1.95,1.87
5494,2023-11-02,Argentina Liga A,21:10,Boca Juniors,Platense,1.36,3.05,153.5,1.83,1.83,-6.5,1.83,1.83


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,22:00,Eua Nba,Utah Jazz,Orlando Magic,222.5,1.91,0.7840,Over
3,14:30,Europa Euroliga,Crvena zvezda,Bayern,157.5,1.91,0.7101,Over
4,14:45,Europa Euroliga,Fenerbahce,Olympiakos,152.5,1.91,0.7164,Over
5,15:00,Europa Euroliga,Zalgiris Kaunas,Lyon-Villeurbanne,161.5,1.91,0.8402,Over
6,16:30,Europa Euroliga,Milano,Monaco,160.5,1.91,0.7438,Over
7,16:30,Europa Euroliga,Valencia,Alba Berlin,160.5,1.91,0.7360,Over
8,16:30,Europa Eurocopa,London Lions,Hapoel Tel-Aviv,172.5,1.91,0.7317,Over
10,05:30,Austrália Nbl,Brisbane Bullets,South East Melbourne,178.5,1.91,0.7329,Over
14,07:00,Coreia Do Sul Kbl,LG Sakers,Goyang Sono,166.5,1.91,0.8417,Over
15,07:00,Coreia Do Sul Kbl,Seoul Thunders,Anyang JungKwanJang,162.5,1.91,0.8318,Over


### AH Zebra V1 (LR)

In [9]:
# Duplicando o dataset
df_ah_pos_v1 = jogos.copy()

# AH
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v1['Zebra'] = df_ah_pos_v1['Zebra'].astype('category')
# Odds
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v1['HA_Odds_H']
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v1['HA_Odds_A']

df_ah_pos_v1.reset_index(drop=True, inplace=True)
df_ah_pos_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v1.empty:
    mdl_ah_pos_v1 = load_model('ML/ah_pos/v1/v1_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v1, df_ah_pos_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] > 1.85))]
    prev = prev[((prev['Over_Line'] >= 200) & (prev['Over_Line'] < 212)) | ((prev['Over_Line'] >= 215) & (prev['Over_Line'] < 218))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V1 (LR) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V1 (LR) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
12,08:35,China Cba,Shandong,Shanxi Zhongyu,-2.5,1.87,0.5601,AH Zebra


### AH Zebra V2 (ADA)

In [10]:
# Duplicando o dataset
df_ah_pos_v2 = jogos.copy()

# AH
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v2['Zebra'] = df_ah_pos_v2['Zebra'].astype('category')
# Odds
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v2['HA_Odds_H']
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v2['HA_Odds_A']

df_ah_pos_v2.reset_index(drop=True, inplace=True)
df_ah_pos_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v2.empty:
    mdl_ah_pos_v2 = load_model('ML/ah_pos/v2/v2_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v2, df_ah_pos_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V2 (ADA) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V2 (ADA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
5,15:00,Europa Euroliga,Zalgiris Kaunas,Lyon-Villeurbanne,-7.5,1.91,0.6337,AH Zebra
7,16:30,Europa Euroliga,Valencia,Alba Berlin,-9.5,1.87,0.6326,AH Zebra
14,07:00,Coreia Do Sul Kbl,LG Sakers,Goyang Sono,-7.5,1.91,0.6337,AH Zebra


### AH Zebra V3 (ADA - SEED 8941)

In [11]:
# Duplicando o dataset
df_ah_pos_v3 = jogos.copy()

# AH
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v3['Zebra'] = df_ah_pos_v3['Zebra'].astype('category')
# Odds
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v3['HA_Odds_H']
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v3['HA_Odds_A']

df_ah_pos_v3.reset_index(drop=True, inplace=True)
df_ah_pos_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v3.empty:
    mdl_ah_pos_v3 = load_model('ML/ah_pos/v3/v3_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v3, df_ah_pos_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### AH Zebra V4 (ADA - SEED 4280)

In [12]:
# Duplicando o dataset
df_ah_pos_v4 = jogos.copy()

# AH
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v4['Zebra'] = df_ah_pos_v4['Zebra'].astype('category')
# Odds
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v4['HA_Odds_H']
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v4['HA_Odds_A']

df_ah_pos_v4.reset_index(drop=True, inplace=True)
df_ah_pos_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v4.empty:
    mdl_ah_pos_v4 = load_model('ML/ah_pos/v3/v4_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v4, df_ah_pos_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
5,15:00,Europa Euroliga,Zalgiris Kaunas,Lyon-Villeurbanne,-7.5,1.91,0.5870,AH Zebra
7,16:30,Europa Euroliga,Valencia,Alba Berlin,-9.5,1.87,0.5860,AH Zebra
14,07:00,Coreia Do Sul Kbl,LG Sakers,Goyang Sono,-7.5,1.91,0.5883,AH Zebra
